In [1]:
# !pip install datasets transformers
import pandas as pd
from datasets import load_dataset
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
# !pip install transformers[sentencepiece]
from transformers import Trainer, TrainingArguments

In [2]:
# pos=pd.read_csv('/content/drive/MyDrive/Added new data 2/positive_add1and2.csv')
# neg=pd.read_csv('/content/drive/MyDrive/Added new data 2/negative_add_1and2.csv')

In [3]:
# pos=pos.drop('Unnamed: 0',axis=1)

In [4]:
# neg

In [5]:
# n=[]
# for i in range(0,115207):
#   n.append(0)

In [6]:
# neg['labels']=n
# neg=neg.drop('label',axis=1)

In [7]:
# neg=neg[0:83000]

In [8]:
# df=neg.append(pos,ignore_index=True)

In [9]:
# df

In [10]:
# h=[]
# for i in df.tweet.values:
#  h.append(i.lower())

In [11]:
# df['tweets']=h

In [12]:
# df=df.drop('tweet',axis=1)

In [13]:
# sub_df=df[0:25000]
# train=sub_df[0:20000]
# test=sub_df[20000:25000]

In [14]:
# df.to_csv('/content/drive/MyDrive/Train and valid spli/df.csv')

In [15]:
# df = df.sample(frac=1).reset_index(drop=True)

In [16]:
# train.to_csv('/content/drive/MyDrive/Train and valid spli/train.csv')
# test.to_csv('/content/drive/MyDrive/Train and valid spli/test.csv')

**XLM-Roberta Casual Language modelling**

In [17]:
path = '/home/chandu/ckm/code/codemix/data/Train and valid spli/'

dataset = load_dataset('csv', data_files={'train':path+'train.csv','test':path+'test.csv'})

Using custom data configuration default-b7e67418bca6bf05
Reusing dataset csv (/home/chandu/.cache/huggingface/datasets/csv/default-b7e67418bca6bf05/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'labels', 'tweets'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'labels', 'tweets'],
        num_rows: 5000
    })
})

In [19]:
model_checkpoint = "xlm-roberta-base"
from transformers import XLMRobertaTokenizerFast
tokenizer =XLMRobertaTokenizerFast.from_pretrained(model_checkpoint, use_fast=True)

In [20]:
def tokenize_function(examples):
    return tokenizer(examples["tweets"], padding=True, truncation=True)

In [21]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2, remove_columns=["tweets","Unnamed: 0","labels"])

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 5000
    })
})

In [23]:
block_size=128
block_size

128

In [24]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [25]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=2,
)

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 5000
    })
})

In [27]:
lm_datasets['train']

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 12874
})

In [28]:
from transformers import XLMRobertaForCausalLM
model = XLMRobertaForCausalLM.from_pretrained(model_checkpoint)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [29]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=4.0
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"]
)

In [31]:
trainer.train()

RuntimeError: CUDA out of memory. Tried to allocate 970.00 MiB (GPU 0; 3.95 GiB total capacity; 2.82 GiB already allocated; 76.94 MiB free; 2.93 GiB reserved in total by PyTorch)

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
import torch

In [ ]:
torch.save(model,'/content/drive/MyDrive/Model/model.pt')

In [ ]:
import torch
new_model=torch.load('/content/drive/MyDrive/Model/model.pt',map_location='cpu')

**XLM-Roberta Masked Language modelling**

In [ ]:
# from transformers import XLMRobertaForMaskedLM
# model2 =XLMRobertaForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
# dataset = load_dataset('csv', data_files={'train':'/content/drive/MyDrive/Train and valid spli/train.csv','valid':'/content/drive/MyDrive/Train and valid spli/valid.csv','test':'/content/drive/MyDrive/Train and valid spli/test.csv'})

In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples["tweets"])
# block_size=128

In [ ]:
# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#         # customize this part to your needs.
#     total_length = (total_length // block_size) * block_size
#     # Split by chunks of max_len.
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

In [ ]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["tweets","Unnamed: 0","labels"])

In [ ]:
# lm_datasets = tokenized_datasets.map(
#     group_texts,
#     batched=True,
#     batch_size=1000,
#     num_proc=4,
# )

In [ ]:
# training_args = TrainingArguments(
#     "test-clm",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-3,
#     weight_decay=0.01,
# )

In [ ]:
# from transformers import DataCollatorForLanguageModeling
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
# trainer2 = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_datasets["train"],
#     eval_dataset=lm_datasets["valid"],
#     data_collator=data_collator,
# )

In [ ]:
# trainer2.train()

In [ ]:
# import math
# eval_results = trainer2.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
47.97